In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Kernel = Microsoft.SemanticKernel.Kernel;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.0.1

## 找出意圖

In [2]:
string request = "I want to send an email to the marketing team celebrating their recent milestone.";

string prompt = $"What is the intent of this request? {request}";

Console.WriteLine(prompt);

What is the intent of this request? I want to send an email to the marketing team celebrating their recent milestone.


In [3]:
Console.WriteLine(await kernel.InvokePromptAsync(prompt));

The intent of this request is to compose an email to congratulate and celebrate the marketing team for reaching a milestone.


## 更明確

In [4]:
prompt = @$"What is the intent of this request? {request}
You can choose between SendEmail, SendMessage, CompleteTask, CreateDocument.";

In [5]:
Console.WriteLine(await kernel.InvokePromptAsync(prompt));

SendEmail


## 增加結構

In [6]:
prompt = @$"Instructions: What is the intent of this request?
Choices: SendEmail, SendMessage, CompleteTask, CreateDocument.
User Input: {request}
Intent: ";

In [7]:
Console.WriteLine(await kernel.InvokePromptAsync(prompt));

SendEmail


In [10]:
prompt = @$"## Instructions
Provide the intent of the request using the following format:

```json
{{
    ""intent"": {{intent}}
}}
```

## Choices
You can choose between the following intents:

```json
[""SendEmail"", ""SendMessage"", ""CompleteTask"", ""CreateDocument""]
```

## User Input
The user input is:

```json
{{
    ""request"": ""{request}""
}}
```

## Intent";

In [11]:
Console.WriteLine(await kernel.InvokePromptAsync(prompt));

{
    "intent": "SendEmail"
}


## 提供情境

以下面的對話為例：


> User: I hate sending emails, no one ever reads them.
>
> AI: I'm sorry to hear that. Messages may be a better way to communicate.
>
> User: I agree, can you send the full status update to the marketing team that way?


如果沒有前後文，那麼會識別為 SendEmail 而不是 SendMessage
因此前後文很重要

In [12]:
string request2 = "can you send the full status update to the marketing team that way?";
string history = @"User input: I hate sending emails, no one ever reads them.
AI response: I'm sorry to hear that. Messages may be a better way to communicate.";

prompt = @$"Instructions: What is the intent of this request?
If you don't know the intent, don't guess; instead respond with ""Unknown"".
Choices: SendEmail, SendMessage, CompleteTask, CreateDocument, Unknown.

User Input: Can you send a very quick approval to the marketing team?
Intent: SendMessage

User Input: Can you send the full update to the marketing team?
Intent: SendEmail

{history}
User Input: {request2}
Intent: ";

In [13]:
Console.WriteLine(await kernel.InvokePromptAsync(prompt));

SendEmail


In [15]:
history = @"<message role=""user"">I hate sending emails, no one ever reads them.</message>
<message role=""assistant"">I'm sorry to hear that. Messages may be a better way to communicate.</message>";

prompt = @$"<message role=""system"">Instructions: What is the intent of this request?
If you don't know the intent, don't guess; instead respond with ""Unknown"".
Choices: SendEmail, SendMessage, CompleteTask, CreateDocument, Unknown.</message>

<message role=""user"">Can you send a very quick approval to the marketing team?</message>
<message role=""system"">Intent:</message>
<message role=""assistant"">SendMessage</message>

<message role=""user"">Can you send the full update to the marketing team?</message>
<message role=""system"">Intent:</message>
<message role=""assistant"">SendEmail</message>

{history}
<message role=""user"">{request2}</message>
<message role=""system"">Intent:</message>";

In [16]:
Console.WriteLine(await kernel.InvokePromptAsync(prompt));

SendEmail
